In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
from mes_fonctions import *

In [3]:
datas = pd.read_csv('movie_metadata_nettoye_1.csv')

## Autres librairies

In [4]:
from sklearn import preprocessing

In [5]:
datas.head()

,id,movie_title,title_year,imdb_score,content_rating,duration,aspect_ratio,color,movie_imdb_link,budget,...,language_Panjabi,language_Tamil,language_Dzongkha,language_Vietnamese,language_Urdu,language_Slovenian,language_Greek,language_Swahili,black_and_white,audience
0,499549,Avatar,2009.0,7.9,PG-13,178.0,1.78,Color,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,237000000.0,...,False,False,False,False,False,False,False,False,False,populaire
1,449088,Pirates of the Caribbean: At World's End,2007.0,7.1,PG-13,169.0,2.35,Color,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,300000000.0,...,False,False,False,False,False,False,False,False,False,populaire
2,2379713,Spectre,2015.0,6.8,PG-13,148.0,2.35,Color,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,245000000.0,...,False,False,False,False,False,False,False,False,False,populaire
3,1345836,The Dark Knight Rises,2012.0,8.5,PG-13,164.0,2.35,Color,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,250000000.0,...,False,False,False,False,False,False,False,False,False,populaire
4,5289954,Star Wars: Episode VII - The Force Awakens ...,NaN,7.1,NaN,NaN,NaN,Color,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,...,False,False,False,False,False,False,False,False,False,NaN


# Ensemble de Groupes de features

In [6]:
l_genre = sorted([ c_state for c_state in datas.columns if re.match('^genre_.*',c_state)])
l_keyw = sorted([ c_state for c_state in datas.columns if re.match('^keyw_.*',c_state)])
l_director = sorted([ c_state for c_state in datas.columns if re.match('^dir_.*',c_state)])
l_acteur = sorted([ c_state for c_state in datas.columns if re.match('^acteur_.*',c_state)])
l_pays = sorted([ c_state for c_state in datas.columns if re.match('^country_.*',c_state)])
l_langue = sorted([ c_state for c_state in datas.columns if re.match('^language_.*',c_state)])
l_black_and_white = ['black_and_white']
l_age = ['age_limite']

# Fonctions Pratiques

In [7]:
def get_movie_id_by_name(m_dataframe=datas, approximativ_movie_name=''):
    m = m_dataframe['movie_title']
    res = []
    for i in m:
        if approximativ_movie_name.lower() in i.lower():
            res.append(i)
    
    for movie_name in res:
        id = m_dataframe[m_dataframe['movie_title']==movie_name]['id'].values[0]
        print("[{}] : {}".format(id, movie_name))

In [8]:
def get_movie_by_id(m_dataframe=datas, movie_id=0):
    movie_id = int(movie_id)
    info_id = m_dataframe[m_dataframe['id']==movie_id]
    title = info_id['movie_title'].values[0]
    act1 = info_id['actor_1_name'].values[0]
    act2 = info_id['actor_2_name'].values[0]
    act3 = info_id['actor_3_name'].values[0]
     
    print("\t\t[{id}] - '{title}'".format(id=movie_id, title=title))

def get_movie_by_id3(m_dataframe=datas, movie_id=0):
    movie_id = int(movie_id)
    info_id = m_dataframe[m_dataframe['id']==movie_id]
    title = info_id['movie_title'].values[0]
    act1 = info_id['actor_1_name'].values[0]
    act2 = info_id['actor_2_name'].values[0]
    act3 = info_id['actor_3_name'].values[0]
    
    id_label = np.nan
    if ('label' in info_id.columns):
        id_label = info_id['label'].values[0]
    
    print("[{id}] - '{title}'\n\t\t\t-> actors [ {act1}, {act2}, {act3} ]".format(
        id=movie_id, title=title, act1=act1, act2=act2, act3=act3))
    
def get_movie_by_id2(m_dataframe=datas, movie_id=0):
    movie_id = int(movie_id)
    title = m_dataframe[m_dataframe['id']==movie_id]['movie_title'].values[0]
    act1 = m_dataframe[m_dataframe['id']==movie_id]['actor_1_name'].values[0]
    act2 = m_dataframe[m_dataframe['id']==movie_id]['actor_2_name'].values[0]
    act3 = m_dataframe[m_dataframe['id']==movie_id]['actor_3_name'].values[0]
    
    print('id [{id}] - title [{title}]\n\t\t\t-> actors [ {act1}, {act2}, {act3} ]'.format(
        id=movie_id, title=title, act1=act1, act2=act2, act3=act3))

In [9]:
def get_movie_id_by_actor(m_dataframe=datas, approximativ_actor_name=''):
    m = m_dataframe['actor_1_name']
    res1 = []
    for i in m[m.notnull()]:
        if approximativ_actor_name.lower() in i.lower():
            res1.append(i)
    
    m = m_dataframe['actor_2_name']
    res2 = []
    for i in m[m.notnull()]:
        if approximativ_actor_name.lower() in i.lower():
            res2.append(i)
    
    m = m_dataframe['actor_3_name']
    res3 = []
    for i in m[m.notnull()]:
        if approximativ_actor_name.lower() in i.lower():
            res3.append(i)
    
    res1 = list(set(res1))
    res2 = list(set(res2))
    res3 = list(set(res3))
    
    for actor_name in res1:
        print('--- {actor}'.format(actor=actor_name))
        id = m_dataframe[m_dataframe['actor_1_name']==actor_name]['id']
        
        for i in id.values:
            get_movie_by_id(m_dataframe, i)
    
    for actor_name in res2:
        print('--- {actor}'.format(actor=actor_name))
        id = m_dataframe[m_dataframe['actor_2_name']==actor_name]['id']
        
        for i in id.values:
            get_movie_by_id(m_dataframe, i)

    for actor_name in res3:
        print('--- {actor}'.format(actor=actor_name))
        id = m_dataframe[m_dataframe['actor_3_name']==actor_name]['id']
        
        for i in id.values:
            get_movie_by_id(m_dataframe, i)

def get_movie_id_by_director(m_dataframe=datas, approximativ_actor_name=''):
    m = m_dataframe['director_name']
    res1 = []
    for i in m[m.notnull()]:
        if approximativ_actor_name.lower() in i.lower():
            res1.append(i)
    
    res1 = list(set(res1))
    
    for director_name in res1:
        print('--- {director}'.format(director=director_name))
        id = m_dataframe[m_dataframe['director_name']==director_name]['id']
        
        for i in id.values:
            get_movie_by_id(m_dataframe, i)




> Les différents modèles testés ont été à partir des K-Means
> 
> Les différences se sont faites sur le nombre d'éléments à prendre en compte
>
> Pour cela nous avons créé une fonction qui est la fonction qui doit ressortir les 
> 5 propositions

In [10]:
def get_proposition_for_id(id_film):
    id_label = datas[datas.id==id_film]['label']
    print("-- Film : {} -- (label :{})".format(datas[datas['id']==id_film]['movie_title'].values[0],
                                               id_label.values[0]))
    for i in id_label:
        u = datas[datas.label==i][['id','imdb_score']]
        u = u.sort_values(['imdb_score'],ascending=False)['id']
        for id in u[:5]:
            get_movie_by_id(datas,int(id))

# MODELE 1 : sur les genres

In [11]:
l_genre = sorted([ c_state for c_state in datas.columns if re.match('^genre_.*',c_state)])
l_keyw = sorted([ c_state for c_state in datas.columns if re.match('^keyw_.*',c_state)])

# La liste des features à prendre en compte pour le clustering
l_my_liste = []
l_my_liste.extend(l_genre)

my_X = datas[l_my_liste]

from sklearn.cluster import KMeans

# Number of clusters
kmeans = KMeans(n_clusters=100)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)

# On ajout les labels des clusters dans le dataframe DATAS
datas['label'] = labels

## Test du modèle :
- annie hall: 75686
- x-men: 120903
- the doors: 101761
- pirates of the caraib: 449088
- Interview with the Vampire: 110148
- toy story: 114709
- Schindler's List: 108052

In [12]:
get_movie_id_by_name(datas,'annie hall')
get_movie_id_by_name(datas,'x-men')
get_movie_id_by_name(datas,'the doors')
get_movie_id_by_name(datas,'pirates of')
get_movie_id_by_name(datas,'Interview with the Vampire')
get_movie_id_by_name(datas,'toy story')
get_movie_id_by_name(datas,"Schindler's List"                    )
get_movie_id_by_name(datas,'lola')

[75686] : Annie Hall
[376994] : X-Men: The Last Stand
[1877832] : X-Men: Days of Future Past
[3385516] : X-Men: Apocalypse
[1270798] : X-Men: First Class
[458525] : X-Men Origins: Wolverine
[290334] : X-Men 2
[120903] : X-Men
[101761] : The Doors
[449088] : Pirates of the Caribbean: At World's End
[383574] : Pirates of the Caribbean: Dead Man's Chest
[1298650] : Pirates of the Caribbean: On Stranger Tides
[325980] : Pirates of the Caribbean: The Curse of the Black Pearl
[110148] : Interview with the Vampire: The Vampire Chronicles
[435761] : Toy Story 3
[120363] : Toy Story 2
[114709] : Toy Story
[108052] : Schindler's List
[130827] : Run Lola Run


In [13]:
get_proposition_for_id(75686)
get_proposition_for_id(120903)
get_proposition_for_id(101761)
get_proposition_for_id(449088)
get_proposition_for_id(110148)
get_proposition_for_id(114709)
get_proposition_for_id(108052)

-- Film : Annie Hall -- (label :18)
		[211915] - 'Amélie'
		[1220617] - 'The Inbetweeners            '
		[53291] - 'Some Like It Hot'
		[25316] - 'It Happened One Night'
		[75686] - 'Annie Hall'
-- Film : X-Men -- (label :7)
		[82971] - 'Raiders of the Lost Ark'
		[118480] - 'Stargate SG-1            '
		[90605] - 'Aliens'
		[372784] - 'Batman Begins'
		[62622] - '2001: A Space Odyssey'
-- Film : The Doors -- (label :16)
		[253474] - 'The Pianist'
		[268978] - 'A Beautiful Mind'
		[80678] - 'The Elephant Man'
		[758758] - 'Into the Wild'
		[2084970] - 'The Imitation Game'
-- Film : Pirates of the Caribbean: At World's End -- (label :15)
		[167260] - 'The Lord of the Rings: The Return of the King'
		[120737] - 'The Lord of the Rings: The Fellowship of the Ring'
		[167261] - 'The Lord of the Rings: The Two Towers'
		[2631186] - 'Baahubali: The Beginning'
		[97576] - 'Indiana Jones and the Last Crusade'
-- Film : Interview with the Vampire: The Vampire Chronicles -- (label :37)
		[110148]

# MODELE 2 : sur les Mots Clefs

In [14]:
l_genre = sorted([ c_state for c_state in datas.columns if re.match('^genre_.*',c_state)])
l_keyw = sorted([ c_state for c_state in datas.columns if re.match('^keyw_.*',c_state)])

# La liste des features à prendre en compte pour le clustering
l_my_liste = []
l_my_liste.extend(l_keyw)


my_X = datas[l_my_liste]

from sklearn.cluster import KMeans

# Number of clusters
kmeans = KMeans(n_clusters=100)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)

# On ajout les labels des clusters dans le dataframe DATAS
datas['label'] = labels

In [15]:
get_proposition_for_id(75686)
get_proposition_for_id(120903)
get_proposition_for_id(101761)
get_proposition_for_id(449088)
get_proposition_for_id(110148)
get_proposition_for_id(114709)
get_proposition_for_id(108052)

-- Film : Annie Hall -- (label :97)
		[75686] - 'Annie Hall'
		[3228904] - 'Empire            '
		[350258] - 'Ray'
		[421082] - 'Control'
		[80549] - 'Coal Miner's Daughter'
-- Film : X-Men -- (label :2)
		[691996] - 'Towering Inferno            '
		[68646] - 'The Godfather'
		[3082898] - 'Kickboxer: Vengeance'
		[92337] - 'Dekalog            '
		[71562] - 'The Godfather: Part II'
-- Film : The Doors -- (label :76)
		[2802850] - 'Fargo            '
		[118715] - 'The Big Lebowski'
		[32976] - 'Rebecca'
		[405159] - 'Million Dollar Baby'
		[378194] - 'Kill Bill: Vol. 2'
-- Film : Pirates of the Caribbean: At World's End -- (label :2)
		[691996] - 'Towering Inferno            '
		[68646] - 'The Godfather'
		[3082898] - 'Kickboxer: Vengeance'
		[92337] - 'Dekalog            '
		[71562] - 'The Godfather: Part II'
-- Film : Interview with the Vampire: The Vampire Chronicles -- (label :2)
		[691996] - 'Towering Inferno            '
		[68646] - 'The Godfather'
		[3082898] - 'Kickboxer: Vengean

# MODELE 3 : On mixte genre et mots clefs

In [16]:
l_genre = sorted([ c_state for c_state in datas.columns if re.match('^genre_.*',c_state)])
l_keyw = sorted([ c_state for c_state in datas.columns if re.match('^keyw_.*',c_state)])

# La liste des features à prendre en compte pour le clustering
l_my_liste = []
l_my_liste.extend(l_genre)
l_my_liste.extend(l_keyw)


my_X = datas[l_my_liste]

from sklearn.cluster import KMeans

# Number of clusters
kmeans = KMeans(n_clusters=100)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)

# On ajout les labels des clusters dans le dataframe DATAS
datas['label'] = labels

In [17]:
get_proposition_for_id(75686)
get_proposition_for_id(120903)
get_proposition_for_id(101761)
get_proposition_for_id(449088)
get_proposition_for_id(110148)
get_proposition_for_id(114709)
get_proposition_for_id(108052)

-- Film : Annie Hall -- (label :34)
		[211915] - 'Amélie'
		[1220617] - 'The Inbetweeners            '
		[53291] - 'Some Like It Hot'
		[45152] - 'Singin' in the Rain'
		[25316] - 'It Happened One Night'
-- Film : X-Men -- (label :49)
		[90605] - 'Aliens'
		[118480] - 'Stargate SG-1            '
		[3498820] - 'Captain America: Civil War'
		[2015381] - 'Guardians of the Galaxy'
		[848228] - 'The Avengers'
-- Film : The Doors -- (label :22)
		[2582802] - 'Whiplash'
		[372824] - 'The Chorus'
		[907657] - 'Once'
		[181875] - 'Almost Famous'
		[358273] - 'Walk the Line'
-- Film : Pirates of the Caribbean: At World's End -- (label :42)
		[167260] - 'The Lord of the Rings: The Return of the King'
		[80684] - 'Star Wars: Episode V - The Empire Strikes Back'
		[120737] - 'The Lord of the Rings: The Fellowship of the Ring'
		[76759] - 'Star Wars: Episode IV - A New Hope'
		[167261] - 'The Lord of the Rings: The Two Towers'
-- Film : Interview with the Vampire: The Vampire Chronicles -- (label :6

In [18]:
#datas.to_csv('movie_metadata_nettoye_2.csv',index=False)

# MODELE 4 : Genre / Mots Clefs / Réalisateurs

In [19]:
l_genre = sorted([ c_state for c_state in datas.columns if re.match('^genre_.*',c_state)])
l_keyw = sorted([ c_state for c_state in datas.columns if re.match('^keyw_.*',c_state)])
l_dir = sorted([ c_state for c_state in datas.columns if re.match('^dir_.*',c_state)])

l_my_liste = []
l_my_liste.extend(l_keyw)
l_my_liste.extend(l_genre)
l_my_liste.extend(l_dir)

my_X = datas[l_my_liste]

from sklearn.cluster import KMeans

# Number of clusters
kmeans = KMeans(n_clusters=100)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)

# ON AJOUTE LES LABEL DES CLUSTERS DANS LES DATAS
datas['label'] = labels

datas.to_csv('movie_metadata_nettoye_2.csv',index=False)

In [20]:
get_proposition_for_id(75686)
get_proposition_for_id(120903)
get_proposition_for_id(101761)
get_proposition_for_id(449088)
get_proposition_for_id(110148)
get_proposition_for_id(114709)
get_proposition_for_id(108052)

-- Film : Annie Hall -- (label :13)
		[1220617] - 'The Inbetweeners            '
		[211915] - 'Amélie'
		[53291] - 'Some Like It Hot'
		[25316] - 'It Happened One Night'
		[107048] - 'Groundhog Day'
-- Film : X-Men -- (label :68)
		[120737] - 'The Lord of the Rings: The Fellowship of the Ring'
		[80684] - 'Star Wars: Episode V - The Empire Strikes Back'
		[76759] - 'Star Wars: Episode IV - A New Hope'
		[167261] - 'The Lord of the Rings: The Two Towers'
		[86190] - 'Star Wars: Episode VI - Return of the Jedi'
-- Film : The Doors -- (label :33)
		[80549] - 'Coal Miner's Daughter'
		[101761] - 'The Doors'
		[1742044] - 'Jersey Boys'
		[443489] - 'Dreamgirls'
		[116250] - 'Evita'
-- Film : Pirates of the Caribbean: At World's End -- (label :68)
		[120737] - 'The Lord of the Rings: The Fellowship of the Ring'
		[80684] - 'Star Wars: Episode V - The Empire Strikes Back'
		[76759] - 'Star Wars: Episode IV - A New Hope'
		[167261] - 'The Lord of the Rings: The Two Towers'
		[86190] - 'Star Wa

# MODELE 5 : Genre / Mots Clefs / Réalisateurs / Acteurs

In [21]:
l_genre = sorted([ c_state for c_state in datas.columns if re.match('^genre_.*',c_state)])
l_keyw = sorted([ c_state for c_state in datas.columns if re.match('^keyw_.*',c_state)])
l_dir = sorted([ c_state for c_state in datas.columns if re.match('^dir_.*',c_state)])
l_acteur = sorted([ c_state for c_state in datas.columns if re.match('^acteur_.*',c_state)])

l_my_liste = []
l_my_liste.extend(l_keyw)
l_my_liste.extend(l_genre)
l_my_liste.extend(l_dir)
l_my_liste.extend(l_acteur)


my_X = datas[l_my_liste]

# Number of clusters
kmeans = KMeans(n_clusters=100)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)

# ON AJOUTE LES LABEL DES CLUSTERS DANS LES DATAS
datas['label'] = labels

#datas.to_csv('movie_metadata_nettoye_2.csv',index=False)

In [22]:
get_proposition_for_id(75686)
get_proposition_for_id(120903)
get_proposition_for_id(101761)
get_proposition_for_id(449088)
get_proposition_for_id(110148)
get_proposition_for_id(114709)
get_proposition_for_id(108052)

-- Film : Annie Hall -- (label :60)
		[75686] - 'Annie Hall'
		[118954] - 'Deconstructing Harry'
		[70707] - 'Sleeper'
		[66808] - 'Bananas'
		[256524] - 'The Curse of the Jade Scorpion'
-- Film : X-Men -- (label :20)
		[1270798] - 'X-Men: First Class'
		[120903] - 'X-Men'
		[3385516] - 'X-Men: Apocalypse'
-- Film : The Doors -- (label :48)
		[108052] - 'Schindler's List'
		[86879] - 'Amadeus'
		[80678] - 'The Elephant Man'
		[758758] - 'Into the Wild'
		[268978] - 'A Beautiful Mind'
-- Film : Pirates of the Caribbean: At World's End -- (label :70)
		[167260] - 'The Lord of the Rings: The Return of the King'
		[80684] - 'Star Wars: Episode V - The Empire Strikes Back'
		[120737] - 'The Lord of the Rings: The Fellowship of the Ring'
		[76759] - 'Star Wars: Episode IV - A New Hope'
		[167261] - 'The Lord of the Rings: The Two Towers'
-- Film : Interview with the Vampire: The Vampire Chronicles -- (label :6)
		[81505] - 'The Shining'
		[63522] - 'Rosemary's Baby'
		[110148] - 'Interview w

# MODELE 6 : Genre / Mots Clefs / Réalisateurs / Acteurs / Black & White / Age Limite

In [23]:
l_genre = sorted([ c_state for c_state in datas.columns if re.match('^genre_.*',c_state)])
l_keyw = sorted([ c_state for c_state in datas.columns if re.match('^keyw_.*',c_state)])
l_dir = sorted([ c_state for c_state in datas.columns if re.match('^dir_.*',c_state)])
l_acteur = sorted([ c_state for c_state in datas.columns if re.match('^acteur_.*',c_state)])
l_black_and_white = ['black_and_white']
l_age = ['age_limite']

l_my_liste = []
l_my_liste.extend(l_keyw)
l_my_liste.extend(l_genre)
l_my_liste.extend(l_dir)
l_my_liste.extend(l_acteur)
l_my_liste.extend(l_black_and_white)
l_my_liste.extend(l_age)

my_X = datas[l_my_liste]

# Number of clusters
kmeans = KMeans(n_clusters=100)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)

# ON AJOUTE LES LABEL DES CLUSTERS DANS LES DATAS
datas['label'] = labels

#datas.to_csv('movie_metadata_nettoye_2.csv',index=False)

In [24]:
get_proposition_for_id(75686)
get_proposition_for_id(120903)
get_proposition_for_id(101761)
get_proposition_for_id(449088)
get_proposition_for_id(110148)
get_proposition_for_id(114709)
get_proposition_for_id(108052)

-- Film : Annie Hall -- (label :30)
		[57012] - 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb'
		[45152] - 'Singin' in the Rain'
		[75686] - 'Annie Hall'
		[85334] - 'A Christmas Story'
		[107048] - 'Groundhog Day'
-- Film : X-Men -- (label :7)
		[1375666] - 'Inception'
		[118480] - 'Stargate SG-1            '
		[3498820] - 'Captain America: Civil War'
		[107290] - 'Jurassic Park'
		[848228] - 'The Avengers'
-- Film : The Doors -- (label :95)
		[99685] - 'Goodfellas'
		[253474] - 'The Pianist'
		[81398] - 'Raging Bull'
		[993846] - 'The Wolf of Wall Street'
		[112641] - 'Casino'
-- Film : Pirates of the Caribbean: At World's End -- (label :94)
		[167260] - 'The Lord of the Rings: The Return of the King'
		[120737] - 'The Lord of the Rings: The Fellowship of the Ring'
		[167261] - 'The Lord of the Rings: The Two Towers'
		[119698] - 'Princess Mononoke'
		[97576] - 'Indiana Jones and the Last Crusade'
-- Film : Interview with the Vampire: The Vampire Chronicles --

# Modèle : un par un 

In [25]:
from sklearn.cluster import KMeans

# ON prend d'abord les films en noir et blanc
mask = (datas['black_and_white']==True)

subset = datas[mask]
l_my_liste = []
l_my_liste.extend(l_keyw)
l_my_liste.extend(l_genre)

my_X = subset[l_my_liste]
kmeans = KMeans(n_clusters=10)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)
datas.loc[mask,'label'] = labels

max_label = datas['label'].max()

# ON prend d'abord les films en couleur pour tous
mask = ((datas['black_and_white']==False) & (datas['age_limite']==0))

subset = datas[mask]
l_my_liste = []
l_my_liste.extend(l_keyw)
l_my_liste.extend(l_genre)

my_X = subset[l_my_liste]
kmeans = KMeans(n_clusters=5)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)
datas.loc[mask,'label'] = labels+max_label
max_label = datas['label'].max()



# ON prend d'abord les films en couleur pour les moins de 7 ans
mask = ((datas['black_and_white']==False) & (datas['age_limite']==7))

subset = datas[mask]
l_my_liste = []
l_my_liste.extend(l_keyw)
l_my_liste.extend(l_genre)

my_X = subset[l_my_liste]
kmeans = KMeans(n_clusters=5)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)
datas.loc[mask,'label'] = labels+max_label
max_label = datas['label'].max()



# ON prend d'abord les films en couleur pour les moins de 14 ans
mask = ((datas['black_and_white']==False) & (datas['age_limite']==14))

subset = datas[mask]
l_my_liste = []
l_my_liste.extend(l_keyw)
l_my_liste.extend(l_genre)

my_X = subset[l_my_liste]
kmeans = KMeans(n_clusters=5)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)
datas.loc[mask,'label'] = labels+max_label
max_label = datas['label'].max()



# ON prend d'abord les films en couleur pour les moins de 17 ans
mask = ((datas['black_and_white']==False) & (datas['age_limite']==17))

subset = datas[mask]
l_my_liste = []
l_my_liste.extend(l_keyw)
l_my_liste.extend(l_genre)

my_X = subset[l_my_liste]
kmeans = KMeans(n_clusters=5)

# Fitting the input data
kmeans = kmeans.fit(my_X)

# Getting the cluster labels
labels = kmeans.predict(my_X)
datas.loc[mask,'label'] = labels+max_label
max_label = datas['label'].max()

# ON sauvegarde
#datas.to_csv('movie_metadata_nettoye_2.csv',index=False)

In [26]:
get_proposition_for_id(75686)
get_proposition_for_id(120903)
get_proposition_for_id(101761)
get_proposition_for_id(449088)
get_proposition_for_id(110148)
get_proposition_for_id(114709)
get_proposition_for_id(108052)

-- Film : Annie Hall -- (label :105)
		[60196] - 'The Good, the Bad and the Ugly'
		[491738] - 'Psych            '
		[68098] - 'M*A*S*H            '
		[70735] - 'The Sting'
		[45152] - 'Singin' in the Rain'
-- Film : X-Men -- (label :110)
		[167260] - 'The Lord of the Rings: The Return of the King'
		[1375666] - 'Inception'
		[120737] - 'The Lord of the Rings: The Fellowship of the Ring'
		[167261] - 'The Lord of the Rings: The Two Towers'
		[119698] - 'Princess Mononoke'
-- Film : The Doors -- (label :115)
		[111161] - 'The Shawshank Redemption'
		[68646] - 'The Godfather'
		[92337] - 'Dekalog            '
		[71562] - 'The Godfather: Part II'
		[110912] - 'Pulp Fiction'
-- Film : Pirates of the Caribbean: At World's End -- (label :110)
		[167260] - 'The Lord of the Rings: The Return of the King'
		[1375666] - 'Inception'
		[120737] - 'The Lord of the Rings: The Fellowship of the Ring'
		[167261] - 'The Lord of the Rings: The Two Towers'
		[119698] - 'Princess Mononoke'
-- Film : Inter

In [27]:
def modelisation_finale(mdata, id):
    # Nous récupérons les id de même label
    film_input = mdata[mdata['id']==id]
    label = film_input['label'].values[0]
    
    director = film_input['director_name'].values[0]
    act1 = film_input['actor_1_name'].values[0]
    act2 = film_input['actor_2_name'].values[0]
    act3 = film_input['actor_3_name'].values[0]
    color = film_input['color'].values[0]
    audience = film_input['audience'].values[0]
    age_limite = film_input['age_limite'].values[0]
    
    # Ensemble des résultats 
    df = mdata[mdata['label']==label]
    my_df = pd.DataFrame({'id':df['id'], 'imdb_score':df['imdb_score']})
    my_df['priorite'] = 0
    
    # Filtrage des informations
    # 1- Ajout pour couleur
    if color=='Black and White':
        tmp = my_df['priorite'] + [2 if i=='Black and White' else 0 for i in df['color']]
        my_df['priorite'] = tmp
        
    # 2- Ajout de l'age limite
    if age_limite<14:
        tmp = my_df['priorite'] + [1 if i==age_limite else 0 for i in df['age_limite']]
        my_df['priorite'] = tmp
     
    # 3- Ajout du réalisateur
    tmp = my_df['priorite'] + [2 if i==director else 0 for i in df['director_name']]
    my_df['priorite'] = tmp
    
    # 4- Ajout des acteurs
    tmp = my_df['priorite'] + [1 if i==act1 else 0 for i in df['actor_1_name']]
    tmp+= [1 if i==act1 else 0 for i in df['actor_2_name']]
    tmp+= [1 if i==act1 else 0 for i in df['actor_3_name']]
    my_df['priorite'] = tmp
    
    tmp = my_df['priorite'] + [1 if i==act2 else 0 for i in df['actor_1_name']]
    tmp+= [1 if i==act2 else 0 for i in df['actor_2_name']]
    tmp+= [1 if i==act2 else 0 for i in df['actor_3_name']]
    my_df['priorite'] = tmp
    
    tmp = my_df['priorite'] + [1 if i==act3 else 0 for i in df['actor_1_name']]
    tmp+= [1 if i==act3 else 0 for i in df['actor_2_name']]
    tmp+= [1 if i==act3 else 0 for i in df['actor_3_name']]
    my_df['priorite'] = tmp
    
    # 5- Ajout de l'audience
    tmp = my_df['priorite'] + [2 if i==audience else 0 for i in df['audience']]
    my_df['priorite'] = tmp
    
    l_id_res = list(my_df.sort_values(['priorite','imdb_score'],ascending=False).head(6)['id'].values)
    l_id_res.remove(id)
    return l_id_res[:5]
    

In [28]:
modelisation_finale(datas, 167261)

[120737, 167260, 1170358, 903624, 2310332]

In [29]:
u = modelisation_finale(datas, 50083)
u

[56193, 281322, 120586, 56592, 47296]